# Ручка 3. Summary

Делает краткое саммари всего конспекта (на вход идет весь файл в обсидиане, можно добавить передачу всей папки)

In [114]:
from langchain.chat_models.gigachat import GigaChat
from dotenv import load_dotenv
import os

load_dotenv()

GIGACHAT_CREDENTIALS = os.environ.get("GIGACHAT_CREDENTIALS")

llm = GigaChat(
    credentials=GIGACHAT_CREDENTIALS,
    model="GigaChat-Pro",
    verify_ssl_certs=True,
    scope="GIGACHAT_API_CORP",
)

In [119]:
# Загружаем конспект
from langchain_community.document_loaders.text import TextLoader

notes_path = "./notes/text.md"

loader = TextLoader(notes_path, "utf-8")
docs = loader.load()

print(docs[0].page_content)

# Тест async на Stuff

In [124]:
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate

# Define prompt
prompt_template = """Write a concise summary of the following:
"{context}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define StuffDocumentsChain
stuff_chain = create_stuff_documents_chain(llm, prompt)


# await stuff_chain.ainvoke({"context": docs})
for chunk in stuff_chain.stream({"context": docs}):
     print(chunk, end="", flush=True)

Влияние параметров пара на показатели комбинированной установки с простым паровым контуром:
- При увеличении давления пара относительный расход пара снижается, что связано с уменьшением количества теплоты в испарительной и перегревательной поверхностях котла.
- С увеличением давления растет температура уходящих газов.
- Располагаемый перепад энтальпий на паровую турбину увеличивается, что приводит к росту степени влажности в конце процесса расширения и снижению КПД проточной части.
- Зависимость КПД парогазовой установки от начального давления достаточно слабая, оптимальное значение увеличивается с ростом температуры газа за газовой турбиной.
- Температура уходящих газов во всем диапазоне изменения параметров остается выше минимально-допустимой.

# Используем Map-Reduce и Refine

In [122]:
from functools import partial
from langchain.schema import  StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain_text_splitters import CharacterTextSplitter

document_prompt = PromptTemplate.from_template("{page_content}")
partial_format_document = partial(format_document, prompt=document_prompt)

map_template = """Ниже представлен список документов
{docs}
На основе этих документов определи, пожалуйста, основные темы 
Полезный ответ:"""

map_prompt = PromptTemplate.from_template(map_template)

map_chain = (
    map_prompt
    | llm
)

collapse_chain = (
    PromptTemplate.from_template("Сделай краткое содержание этих документов по основным темам:\n\n{context}")
    | llm
)

reduce_chain = (
    PromptTemplate.from_template("Объедени эти краткие содержания в одно целое:\n\n{context}")
    | llm
)

map_reduce = (map_chain.map() | collapse_chain | reduce_chain | StrOutputParser())

In [123]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

await map_reduce.ainvoke(split_docs)

'Данные документы охватывают такие основные темы, как влияние различных параметров на мощность паровой турбины, зависимость коэффициента полезного действия парогазовой установки от начального давления, поиск оптимальных параметров для создания высокоэффективного оборудования, использование тепла уходящих газов, влияние минимального температурного напора на параметры установки и, наконец, влияние параметров пара на работу комбинированной установки с простым паровым контуром.'

In [108]:
docs *= 2

In [109]:
len(docs)

64

In [110]:
sum([len(el.page_content) for el in docs])

180736